<a href="https://colab.research.google.com/github/markoo26/thehappymountain/blob/main/Experimentation_(Part_5_Bayesian_Optimisation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title 🚚 Imports

import numpy as np
import scipy
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [4]:
#@title 🔧 Functions

def jit_plus_server(parameters):
    """Simulates server CPU time with a complex cost landscape containing two peaks."""
    x = np.array(parameters)
    d = len(x)
    x1 = x - 0.15*np.ones(shape=(d,))
    x2 = x - 0.85*np.ones(shape=(d,))
    cpu_time = 2 - np.exp(-10*x1**2) - 0.5*np.exp(-10*x2**2)
    return cpu_time.mean() + .005*np.random.normal()

class GPR_viz: # same as GPR4
    """
    Full Gaussian Process Regression with uncertainty quantification.
    Same implementation as GPR4 - returns both prediction and uncertainty estimates.
    """
    def __init__(self, parameters, measurements, sigma):
        self.x = parameters
        self.y = np.array(measurements)
        self.sigma = sigma

        self.mean_y = self.y.mean()
        if len(self.y) > 1:
            self.std_y = self.y.std()
        else:
            self.std_y = 1

        self.y -= self.mean_y

    def kernel(self, x1, x2):
        distance_squared = ((x1-x2)**2).sum()
        return np.exp( -distance_squared/(2*self.sigma**2) )

    def estimate(self, query_parameter):
        kernels_x_query = np.array([
            self.kernel(x, query_parameter)
            for x in self.x
        ])
        kernels_x_x = np.array([
            [
                self.kernel(x1, x2)
                for x1 in self.x
            ]
            for x2 in self.x
        ])

        weights = kernels_x_query.T @ np.linalg.pinv(kernels_x_x)
        expectation = self.mean_y + weights @ self.y
        uncertainty_squared = 1 - weights @ kernels_x_query
        return expectation, self.std_y*np.sqrt(np.maximum(0, uncertainty_squared))

class GPR1:
    """
    Baseline predictor - always returns the mean of observed measurements.
    No spatial correlation or kernel function used.
    """
    def __init__(self, parameters, measurements):
        self.x = parameters
        self.y = np.array(measurements)
        self.mean_y = self.y.mean()

    def estimate(self, query_parameter):
        return self.mean_y


class GPR2:
    """
    Weighted average GPR using normalized kernel weights.
    Returns predictions but no uncertainty estimates.
    """
    def __init__(self, parameters, measurements, sigma):
        self.x = parameters
        self.y = np.array(measurements)
        self.sigma = sigma

        self.mean_y = self.y.mean()
        self.y -= self.mean_y

    def kernel(self, x1, x2):
        distance_squared = ((x1-x2)**2).sum()
        return np.exp( -distance_squared/(2*self.sigma**2) )

    def estimate(self, query_parameter):
        weights = [
            self.kernel(x, query_parameter)
            for x in self.x
        ]
        weights = np.array(weights)
        weights = weights / weights.sum()
        return self.mean_y + weights @ self.y

class GPR3:
    """
    GPR with proper kernel matrix inversion using linalg.inv.
    Returns predictions but no uncertainty estimates.
    May fail with singular matrices.
    """
    def __init__(self, parameters, measurements, sigma):
        self.x = parameters
        self.y = np.array(measurements)
        self.sigma = sigma

        self.mean_y = self.y.mean()
        self.y -= self.mean_y

    def kernel(self, x1, x2):
        distance_squared = ((x1-x2)**2).sum()
        return np.exp( -distance_squared/(2*self.sigma**2) )

    def estimate(self, query_parameter):
        kernels_x_query = np.array([
            self.kernel(x, query_parameter)
            for x in self.x
        ])
        kernels_x_x = np.array([
            [
                self.kernel(x1, x2)
                for x1 in self.x
            ]
            for x2 in self.x
        ])

        weights = kernels_x_query.T @ np.linalg.inv(kernels_x_x)
        return self.mean_y + weights @ self.y


class GPR4:
    """
    Full GPR with uncertainty quantification using pseudoinverse.
    Returns both prediction mean and uncertainty (standard deviation).
    More numerically stable than GPR3 due to pinv instead of inv.
    """
    def __init__(self, parameters, measurements, sigma):
        self.x = parameters
        self.y = np.array(measurements)
        self.sigma = sigma

        self.mean_y = self.y.mean()
        if len(self.y) > 1:
            self.std_y = self.y.std()
        else:
            self.std_y = 1

        self.y -= self.mean_y

    def kernel(self, x1, x2):
        distance_squared = ((x1-x2)**2).sum()
        return np.exp( -distance_squared/(2*self.sigma**2) )

    def estimate(self, query_parameter):
        kernels_x_query = np.array([
            self.kernel(x, query_parameter)
            for x in self.x
        ])
        kernels_x_x = np.array([
            [
                self.kernel(x1, x2)
                for x1 in self.x
            ]
            for x2 in self.x
        ])

        weights = kernels_x_query.T @ np.linalg.pinv(kernels_x_x)
        expectation = self.mean_y + weights @ self.y
        uncertainty_squared = 1 - weights @ kernels_x_query
        uncertainty = np.sqrt(uncertainty_squared)
        return expectation, self.std_y*uncertainty

def evaluate(gpr, x):
    """Evaluates lower confidence bound (LCB) at parameter x using the GPR model."""
    x = np.mod(x, 1)
    y, sigma_y = gpr.estimate(x)
    lcb = y - sigma_y
    return x, lcb

def random_search(gpr, num_parameters, num_iterations=1000):
    """
    Finds minimum LCB via random walk optimization.
    Returns best parameters found and trace of LCB values.
    """
    step_size = 0.1
    x_current = np.random.normal(size=num_parameters)
    x_current, lcb_current = evaluate(gpr, x_current)
    trace = []
    for _ in range(num_iterations):
        x_test = (
            x_current
            + step_size*np.random.normal(size=num_parameters)
        )
        x_test, lcb_test = evaluate(gpr, x_test)
        if lcb_test < lcb_current:
            lcb_current = lcb_test
            x_current = x_test
        trace.append( lcb_current )
    return x_current, np.array(trace)

class BayesianOptimizer:
    """
    Bayesian optimization framework that suggests new parameters to evaluate
    by minimizing the lower confidence bound of a GPR4 surrogate model.
    """
    def __init__(self, num_parameters):
        self.num_parameters = num_parameters
        self.parameters = []
        self.measurements = []
        self.x0 = np.array([0.5]*num_parameters)

    def ask(self):
        if len(self.measurements)==0:
            return self.x0
        return self.new_parameter()

    def new_parameter(self):
        gpr = GPR4(
            self.parameters,
            self.measurements,
            sigma=0.15,
        )
        return random_search(gpr, self.num_parameters)[0]

    def tell(self, parameter, measurement):
        self.parameters.append(parameter)
        self.measurements.append(measurement)

In [5]:
#@title 🌱 Random seed setup

np.random.seed(42)

In [6]:
#@title 🎲 Example run
parameter0 = 0.5
cpu_time0 = jit_plus_server([parameter0])

print(f"CPU time for parameter0: {parameter0}:  {cpu_time0}")


CPU time for parameter0: 0.5:  1.561847020279757


In [7]:
#@title 🕰️ Example Gaussian Process Regression for sample parameters

x = np.array([parameter0])
y = np.array([cpu_time0])

gpr = GPR_viz(x, y, sigma=0.15)
x_hats = np.linspace(0,1,100)
y_hats = []

sigma_y_hats = []
for x_hat in x_hats:
    ret = gpr.estimate(x_hat)
    try:
        y_hat, sigma_y_hat = ret
    except:
        y_hat = ret
        sigma_y_hat = 0
    y_hats.append(y_hat)
    sigma_y_hats.append(sigma_y_hat)

y_hats = np.array(y_hats)
sigma_y_hats = np.array(sigma_y_hats)


df = pd.DataFrame({
    "x_hats": x_hats,
    "y_hats": y_hats,
    "sigma_y_hats": sigma_y_hats
})

df[f'y_hats_upper'] = df['y_hats'] + df['sigma_y_hats']
df[f'y_hats_lower'] = df['y_hats'] - df['sigma_y_hats']


df_means = df.melt(id_vars=['x_hats'], value_vars=['y_hats'])

fig = px.line(df_means, x='x_hats', y='value', color='variable', markers=True)

fig.add_traces([
    go.Scatter(
        x=df["x_hats"],
        y=df["y_hats_upper"],
        fill='tonexty',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name="95% AB Testing CI"
    )
])

fig.add_traces([
    go.Scatter(
        x=df["x_hats"],
        y=df["y_hats_lower"],
        fill='tonexty',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name="95% AB Testing CI"
    )
])

df = pd.DataFrame({'parameter': [parameter0], 'cpu_time': [cpu_time0], 'size': [20] })
fig_sc = px.scatter(df, x='parameter', y='cpu_time', size='size')
fig_sc.update_traces(marker=dict(color='red'))

fig = go.Figure(data=fig.data + fig_sc.data)

fig.show()

In [8]:
#@title ⚇ Gaussian Process Regression for 2 points


parameter1 = 0
cpu_time1 = jit_plus_server([parameter1])
print(cpu_time1)


x = np.array([parameter0, parameter1])
y = np.array([cpu_time0, cpu_time1])

gpr = GPR_viz(x, y, sigma=0.15)
x_hats = np.linspace(0,1,100)
y_hats = []

sigma_y_hats = []
for x_hat in x_hats:
    ret = gpr.estimate(x_hat)
    try:
        y_hat, sigma_y_hat = ret
    except:
        y_hat = ret
        sigma_y_hat = 0
    y_hats.append(y_hat)
    sigma_y_hats.append(sigma_y_hat)

y_hats = np.array(y_hats)
sigma_y_hats = np.array(sigma_y_hats)

df = pd.DataFrame({
    "x_hats": x_hats,
    "y_hats": y_hats,
    "sigma_y_hats": sigma_y_hats
})

df[f'y_hats_upper'] = df['y_hats'] + df['sigma_y_hats']
df[f'y_hats_lower'] = df['y_hats'] - df['sigma_y_hats']


df_means = df.melt(id_vars=['x_hats'], value_vars=['y_hats'])

fig = px.line(df_means, x='x_hats', y='value', color='variable', markers=True)

fig.add_traces([
    go.Scatter(
        x=df["x_hats"],
        y=df["y_hats_upper"],
        fill='tonexty',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name="95% AB Testing CI"
    )
])

fig.add_traces([
    go.Scatter(
        x=df["x_hats"],
        y=df["y_hats_lower"],
        fill='tonexty',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name="95% AB Testing CI"
    )
])

df = pd.DataFrame({'parameter': [parameter0, parameter1], 'cpu_time': [cpu_time0, cpu_time1], 'size': [20, 20] })
fig_sc = px.scatter(df, x='parameter', y='cpu_time', size='size')
fig_sc.update_traces(marker=dict(color='red'))

fig = go.Figure(data=fig.data + fig_sc.data)

fig.add_vline(x=0, line_dash='dot')
fig.add_vline(x=1, line_dash='dot')

fig.add_annotation(
    x=0,
    ax=50,
    y=1.3,
    text="minimize CPU time",
    font=dict(size=12, color="black"),
    showarrow=True,
    arrowhead=3
)

fig.add_annotation(
    x=1,
    ax=50,
    y=1.5,
    text="maximize model uncertainty",
    font=dict(size=12, color="black"),
    showarrow=True,
    arrowhead=3
)

fig.show()


1.2004283834652223


In [9]:
#@title ⚖️ Gaussian Process regression with balancing the tradeoff between two parameters

x = np.array([parameter0, parameter1])
y = np.array([cpu_time0, cpu_time1])

gpr = GPR_viz(x, y, sigma=0.15)
x_hats = np.linspace(0,1,100)
y_hats = []

sigma_y_hats = []
for x_hat in x_hats:
    ret = gpr.estimate(x_hat)
    try:
        y_hat, sigma_y_hat = ret
    except:
        y_hat = ret
        sigma_y_hat = 0
    y_hats.append(y_hat)
    sigma_y_hats.append(sigma_y_hat)

y_hats = np.array(y_hats)
sigma_y_hats = np.array(sigma_y_hats)

df = pd.DataFrame({
    "x_hats": x_hats,
    "y_hats": y_hats,
    "sigma_y_hats": sigma_y_hats
})

df[f'y_hats_upper'] = df['y_hats'] + df['sigma_y_hats']
df[f'y_hats_lower'] = df['y_hats'] - df['sigma_y_hats']


df_means = df.melt(id_vars=['x_hats'], value_vars=['y_hats'])

fig = px.line(df_means, x='x_hats', y='value', color='variable', markers=True)

fig.add_traces([
    go.Scatter(
        x=df["x_hats"],
        y=df["y_hats_upper"],
        fill='tonexty',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name="95% AB Testing CI"
    )
])

fig.add_traces([
    go.Scatter(
        x=df["x_hats"],
        y=df["y_hats_lower"],
        fill='tonexty',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name="95% AB Testing CI"
    )
])

df_sc = pd.DataFrame({'parameter': [parameter0, parameter1], 'cpu_time': [cpu_time0, cpu_time1], 'size': [20, 20] })
fig_sc = px.scatter(df_sc, x='parameter', y='cpu_time', size='size')
fig_sc.update_traces(marker=dict(color='red'))

fig = go.Figure(data=fig.data + fig_sc.data)

fig.add_vline(x=0, line_dash='dot')
fig.add_vline(x=1, line_dash='dot')

fig.add_annotation(
    x=0,
    ax=50,
    y=1.3,
    text="minimize CPU time",
    font=dict(size=12, color="black"),
    showarrow=True,
    arrowhead=3
)

fig.add_annotation(
    x=1,
    ax=50,
    y=1.5,
    text="maximize model uncertainty",
    font=dict(size=12, color="black"),
    showarrow=True,
    arrowhead=3
)

fig2 = px.line(df, x='x_hats', y='y_hats_lower', markers=False)

fig = go.Figure(data=fig.data + fig2.data)

fig.add_vline(x=0.111111111, line_dash='dot')

fig.add_annotation(
    x=0.1111,
    ax=50,
    y=1.3,
    text="balance CPU time and model uncertainty",
    font=dict(size=12, color="black"),
    showarrow=True,
    arrowhead=3
)
fig.show()

In [10]:
#@title 🌐 Gaussian Process Regression extended with additional four optimization rounds
x = np.array([parameter0, parameter1])
y = np.array([cpu_time0, cpu_time1])


for opt_round, params in enumerate([(.11, 1.0036), (.2020, 1.02231), (1.0, 1.598), (.1515, .99696)]):
  x = np.append(x, params[0])
  y = np.append(y, params[1])

  gpr = GPR_viz(x, y, sigma=0.15)
  x_hats = np.linspace(0,1,100)

  y_hats = []

  sigma_y_hats = []
  for x_hat in x_hats:
      ret = gpr.estimate(x_hat)
      try:
          y_hat, sigma_y_hat = ret
      except:
          y_hat = ret
          sigma_y_hat = 0
      y_hats.append(y_hat)
      sigma_y_hats.append(sigma_y_hat)

  y_hats = np.array(y_hats)
  sigma_y_hats = np.array(sigma_y_hats)

  df = pd.DataFrame({
      "x_hats": x_hats,
      "y_hats": y_hats,
      "sigma_y_hats": sigma_y_hats
  })


  df[f'y_hats_upper'] = df['y_hats'] + df['sigma_y_hats']
  df[f'y_hats_lower'] = df['y_hats'] - df['sigma_y_hats']


  df_means = df.melt(id_vars=['x_hats'], value_vars=['y_hats'])

  fig = px.line(df_means, x='x_hats', y='value', color='variable', markers=True)

  fig.add_traces([
      go.Scatter(
          x=df["x_hats"],
          y=df["y_hats_upper"],
          fill='tonexty',
          fillcolor='rgba(0, 0, 255, 0.2)',
          line=dict(color='rgba(255,255,255,0)'),
          hoverinfo="skip",
          showlegend=True,
          name="95% AB Testing CI"
      )
  ])

  fig.add_traces([
      go.Scatter(
          x=df["x_hats"],
          y=df["y_hats_lower"],
          fill='tonexty',
          fillcolor='rgba(0, 0, 255, 0.2)',
          line=dict(color='rgba(255,255,255,0)'),
          hoverinfo="skip",
          showlegend=True,
          name="95% AB Testing CI"
      )
  ])

  df = pd.DataFrame({'parameter': x, 'cpu_time': y, 'size': [20] * len(x) })
  fig_sc = px.scatter(df, x='parameter', y='cpu_time', size='size')
  fig_sc.update_traces(marker=dict(color='red'))

  fig = go.Figure(data=fig.data + fig_sc.data)
  fig.update_layout(title=f"Round of optimization: {opt_round+1}. Proposed param: {params[0]}")
  fig.show()



In [11]:
#@title 💥 Plot exponential weights
x = np.linspace(0, 1, 100)

df = pd.DataFrame({'x': x, 'weight': np.exp(-x**2 / .1), 'line_dash': ['dot']*100})
px.line(df, x='x', y='weight', line_dash='line_dash')

In [12]:
#@title 🚓 GPR 2 with distance weight and cluster downweighting visualization

parameters = np.array([0.5, 0.0])
measurements = [1.52, 1.21]
gpr2 = GPR2(parameters, measurements, sigma=0.25)

print(f"Estimates for GPR2 for 0.25: {gpr2.estimate(0.25)} and 0.4: {gpr2.estimate(0.4)}")

parameters = np.array([0.5, 0.0, 0.4])
measurements = [1.52, 1.21, gpr2.estimate(0.4)]
gpr2a = GPR2(parameters, measurements, sigma=0.25)
print (gpr2a.estimate(0.25))

df = pd.DataFrame({'parameters': parameters, 'measurements':measurements, 'symbol': 'measurements'})
df.loc[len(df)] = [0.25, gpr2.estimate(0.25), 'distance weights and cluster downweighting']
df.loc[len(df)] = [0.25, gpr2a.estimate(0.25), 'distance weights']
fig = px.scatter(df, x='parameters', y='measurements', color='symbol', symbol='symbol')
fig.show()

Estimates for GPR2 for 0.25: 1.365 and 0.4: 1.4482426828846955
1.3989447654272769


In [13]:
#@title 🚓 Visualization for GPR3 process
gpr3 = GPR3(parameters, measurements, sigma=0.15)
x_hats = np.linspace(0,1,100)
y_hats = [gpr3.estimate(x_hat) for x_hat in x_hats]

df = pd.DataFrame({'x_hats': x_hats, 'y_hats': y_hats})

fig = px.line(df, x='x_hats', y='y_hats')

parameters = np.array([0.5, 0.0])
measurements = [1.52, 1.21]
df2 = pd.DataFrame({'parameters': parameters, 'measurements':measurements,'size': [20] * 2})
fig2 = px.scatter(df2, x='parameters', y='measurements', size='size')
fig2.update_traces(marker=dict(color='red'))
fig3 = go.Figure(data=fig.data + fig2.data)
fig3.show()

In [14]:
#@title 🧑‍🤝‍🧑 Different GPR functions plot

x = np.random.uniform(size=(3,))

x0 = np.random.uniform(size=(3,))
y0 = 1 - (x0-0.4)**2

x1 = np.random.uniform(size=(4,))
y1 = y = (x1-0.7)**2 + 2*x1**3

x2 = np.random.uniform(size=(4,))
y2 = 1 - (x2-0.3)**2 + .25*np.sin(10*x2)

x3 = np.random.uniform(size=(5,))
y3 = 1 - (x3-0.9)**2 + .1*np.sin(10*(x3-0.2))

x_and_y = {0: [x0, y0], 1: [x1, y1], 2: [x2, y2], 3: [x3, y3]}

for opt_round in range(4):
  x = x_and_y[opt_round][0]
  y = x_and_y[opt_round][1]

  gpr = GPR_viz(x, y, sigma=0.15)
  x_hats = np.linspace(0,1,100)

  y_hats = []

  sigma_y_hats = []
  for x_hat in x_hats:
      ret = gpr.estimate(x_hat)
      try:
          y_hat, sigma_y_hat = ret
      except:
          y_hat = ret
          sigma_y_hat = 0
      y_hats.append(y_hat)
      sigma_y_hats.append(sigma_y_hat)

  y_hats = np.array(y_hats)
  sigma_y_hats = np.array(sigma_y_hats)

  df = pd.DataFrame({
      "x_hats": x_hats,
      "y_hats": y_hats,
      "sigma_y_hats": sigma_y_hats
  })


  df[f'y_hats_upper'] = df['y_hats'] + df['sigma_y_hats']
  df[f'y_hats_lower'] = df['y_hats'] - df['sigma_y_hats']


  df_means = df.melt(id_vars=['x_hats'], value_vars=['y_hats'])

  fig = px.line(df_means, x='x_hats', y='value', color='variable', markers=True)

  fig.add_traces([
      go.Scatter(
          x=df["x_hats"],
          y=df["y_hats_upper"],
          fill='tonexty',
          fillcolor='rgba(0, 0, 255, 0.2)',
          line=dict(color='rgba(255,255,255,0)'),
          hoverinfo="skip",
          showlegend=True,
          name="95% AB Testing CI"
      )
  ])

  fig.add_traces([
      go.Scatter(
          x=df["x_hats"],
          y=df["y_hats_lower"],
          fill='tonexty',
          fillcolor='rgba(0, 0, 255, 0.2)',
          line=dict(color='rgba(255,255,255,0)'),
          hoverinfo="skip",
          showlegend=True,
          name="95% AB Testing CI"
      )
  ])

  df = pd.DataFrame({'parameter': x, 'cpu_time': y, 'size': [20] * len(x) })
  fig_sc = px.scatter(df, x='parameter', y='cpu_time', size='size')
  fig_sc.update_traces(marker=dict(color='red'))

  fig = go.Figure(data=fig.data + fig_sc.data)
  fig.update_layout(title=f"Round of optimization: {opt_round+1}. Proposed param: {params[0]}")
  fig.show()



In [15]:
#@title 🎯 All 7 compiler params optimization

parameters = [ np.array([0.5]), np.array([0.0]) ]
measurements = [1.52, 1.21]

gpr4 = GPR4(parameters, measurements, sigma=0.15)
x_opt, trace = random_search(gpr4, 1)

df = pd.DataFrame({'x': list(range(len(trace))), 'trace': trace})
px.line(df, x='x', y='trace', markers=True)

In [16]:
#@title 🏗️ Complete Bayesian optimizer run

bo = BayesianOptimizer(num_parameters=7)
trace = []
cpu_time = None
i = 0

bo = BayesianOptimizer(num_parameters=7)
trace = []
for _ in range(48):
    parameter = bo.ask()
    cpu_time = jit_plus_server(parameter)
    bo.tell(parameter, cpu_time)
    trace.append(cpu_time)

df = pd.DataFrame({'x': list(range(len(trace))), 'trace': trace})
px.line(df, x='x', y='trace')